# Medical Q&A Assistant
### Week 2 Exercise — Stella Oiro (Andela AI Engineering Bootcamp)

Clinical AI assistant for healthcare professionals and medical students.

**Features:**
- Gradio chat UI with streaming responses
- Switch between GPT and Claude
- Tool: Drug information lookup from a local SQLite database
- Voice responses (text-to-speech)
- Example questions to get started

In [ ]:
import os
import json
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (optional)")

In [ ]:
openai_client = OpenAI()
anthropic_client = OpenAI(
    api_key=anthropic_api_key,
    base_url="https://api.anthropic.com/v1/"
)

GPT_MODEL = "gpt-4.1-mini"
CLAUDE_MODEL = "claude-haiku-4-5"

print("Clients ready.")

In [ ]:
DB = "drug_info.db"

drugs = [
    ("metformin",    "Biguanide",            "Type 2 diabetes",                      "500mg twice daily with meals, up to 2000mg/day",    "Renal impairment (eGFR<30), contrast media",  "GI upset, lactic acidosis (rare)"),
    ("amoxicillin",  "Penicillin antibiotic", "Bacterial infections, pneumonia, UTI", "500mg three times daily for 5-7 days",              "Penicillin allergy",                          "Diarrhoea, rash, nausea"),
    ("lisinopril",   "ACE inhibitor",         "Hypertension, heart failure",          "2.5-10mg once daily, up to 40mg/day",               "Pregnancy, renal artery stenosis",            "Dry cough, hyperkalaemia, hypotension"),
    ("atorvastatin", "Statin",                "Hyperlipidaemia, cardiovascular risk", "10-80mg once daily at night",                       "Active liver disease, pregnancy",             "Myopathy, raised liver enzymes"),
    ("salbutamol",   "Beta-2 agonist",        "Asthma, COPD bronchospasm",            "100-200mcg inhaled as needed, up to 4x daily",      "Hypersensitivity",                            "Tremor, tachycardia"),
    ("paracetamol",  "Analgesic/antipyretic", "Mild to moderate pain, fever",         "500-1000mg every 4-6 hours, max 4g/day",            "Severe hepatic impairment",                   "Hepatotoxicity in overdose"),
    ("omeprazole",   "Proton pump inhibitor", "GORD, peptic ulcer, H. pylori",        "20-40mg once daily before food",                    "Hypersensitivity to PPIs",                    "Headache, GI upset"),
    ("warfarin",     "Anticoagulant",         "AF, DVT, PE, mechanical heart valves", "Individualised based on INR target (usually 2-3)", "Active bleeding, pregnancy",                  "Bleeding, skin necrosis (rare)"),
]

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS drugs (
            name TEXT PRIMARY KEY, class TEXT, indications TEXT,
            adult_dose TEXT, contraindications TEXT, side_effects TEXT
        )
    ''')
    cursor.executemany('INSERT OR REPLACE INTO drugs VALUES (?,?,?,?,?,?)', drugs)
    conn.commit()

print(f"Database ready with {len(drugs)} drugs.")

In [ ]:
def get_drug_info(drug_name):
    print(f"Tool called: {drug_name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM drugs WHERE name = ?', (drug_name.lower(),))
        row = cursor.fetchone()
    if row:
        return (
            f"Drug: {row[0].title()}\n"
            f"Class: {row[1]}\n"
            f"Indications: {row[2]}\n"
            f"Adult dose: {row[3]}\n"
            f"Contraindications: {row[4]}\n"
            f"Side effects: {row[5]}"
        )
    return f"No information found for '{drug_name}'."

tools = [{
    "type": "function",
    "function": {
        "name": "get_drug_info",
        "description": "Look up dosage, indications, contraindications and side effects for a drug.",
        "parameters": {
            "type": "object",
            "properties": {
                "drug_name": {"type": "string", "description": "Generic drug name e.g. metformin"}
            },
            "required": ["drug_name"],
            "additionalProperties": False
        }
    }
}]

system_message = """
You are a clinical knowledge assistant with the expertise of a senior physician.
When asked about a specific drug, always use the get_drug_info tool.
Respond in markdown. Remind users that clinical decisions require a qualified professional.
"""

print("Tools and system prompt ready.")

In [ ]:
def speak(text):
    response = openai_client.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=text[:600]
    )
    return response.content


def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        args = json.loads(tool_call.function.arguments)
        result = get_drug_info(args["drug_name"])
        responses.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
    return responses


def user_turn(message, history):
    """Add user message to history and clear the textbox."""
    return "", history + [{"role": "user", "content": message}]


def bot_turn(history, model_choice):
    """Generate streaming assistant response with tool calling and audio."""
    user_message = history[-1]["content"]
    prior = [{"role": h["role"], "content": h["content"]} for h in history[:-1]]

    client = anthropic_client if model_choice == "Claude" else openai_client
    model  = CLAUDE_MODEL    if model_choice == "Claude" else GPT_MODEL

    messages = [{"role": "system", "content": system_message}] + prior + [{"role": "user", "content": user_message}]

    # Tool-calling loop
    response = client.chat.completions.create(model=model, messages=messages, tools=tools)
    while response.choices[0].finish_reason == "tool_calls":
        tool_msg = response.choices[0].message
        tool_responses = handle_tool_calls(tool_msg)
        messages.append(tool_msg)
        messages.extend(tool_responses)
        response = client.chat.completions.create(model=model, messages=messages, tools=tools)

    # Stream the reply
    stream = client.chat.completions.create(model=model, messages=messages, stream=True)
    reply = ""
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ""
        yield history + [{"role": "assistant", "content": reply}], None

    # Generate audio after full reply
    audio = speak(reply)
    yield history + [{"role": "assistant", "content": reply}], audio


print("Functions ready.")

In [ ]:
with gr.Blocks(title="Medical Q&A Assistant") as ui:
    gr.Markdown("""
    # 🏥 Medical Q&A Assistant
    **Ask clinical questions — drug dosages, conditions, guidelines.**
    *For educational purposes. Always verify with a qualified clinician.*
    """)

    model_dd = gr.Dropdown(choices=["GPT", "Claude"], value="GPT", label="Select Model")

    chatbot = gr.Chatbot(height=450, type="messages", placeholder="Ask me about any drug, condition or guideline...")

    audio_out = gr.Audio(label="Voice Response", autoplay=True)

    with gr.Row():
        msg = gr.Textbox(label="Your question:", placeholder="e.g. What is the dose of metformin?", scale=9)
        send_btn = gr.Button("Send ➤", variant="primary", scale=1)

    gr.Examples(
        examples=[
            ["What is the adult dose of amoxicillin for a chest infection?", "GPT"],
            ["What are the contraindications of warfarin?",                  "Claude"],
            ["Explain the difference between Type 1 and Type 2 diabetes",   "GPT"],
            ["What are the NICE guidelines for hypertension management?",    "GPT"],
            ["What are the early signs of sepsis?",                          "Claude"],
        ],
        inputs=[msg, model_dd]
    )

    # Both Enter key and Send button trigger the same flow
    gr.on(
        triggers=[msg.submit, send_btn.click],
        fn=user_turn,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    ).then(
        bot_turn,
        inputs=[chatbot, model_dd],
        outputs=[chatbot, audio_out]
    )

ui.launch()